<a href="https://colab.research.google.com/github/kueyO/DataScience/blob/main/CoalSupplyModule9LinearOpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install scipy pandas
from scipy.optimize import linprog
import pandas as pd

# Data from the problem
costs = [49.50, 50.00, 61.00, 63.50, 66.50, 71.00, 72.50, 80.00]  # $/ton
capacities = [300, 600, 510, 655, 575, 680, 450, 490]  # mtons
volatilities = [15, 16, 18, 20, 21, 22, 23, 25]  # %
union_flags = [1, 1, 0, 1, 0, 1, 0, 0]  # 1 = Union mine
rail_flags = [1, 0, 1, 0, 0, 0, 1, 1]  # 1 = Rail delivery
truck_flags = [0, 1, 0, 1, 1, 1, 0, 0]  # 1 = Truck delivery

total_demand = 1225  # mtons
volatility_req = 19  # %
union_minimum = 0.5 * total_demand  # mtons
rail_capacity = 650  # mtons
truck_capacity = 720  # mtons

# Linear programming setup
c = costs  # Minimize cost

# Coefficients for constraints
A = [
    [-v for v in volatilities],  # Average volatility constraint
    [u for u in union_flags],    # Union coal constraint
    [r for r in rail_flags],     # Rail capacity constraint
    [t for t in truck_flags],    # Truck capacity constraint
    [1] * len(costs)             # Total quantity constraint
]
b = [
    -volatility_req * total_demand,  # RHS for volatility
    union_minimum,                  # Minimum union requirement
    rail_capacity,                  # Rail capacity
    truck_capacity,                 # Truck capacity
    total_demand                    # Total demand
]

# Upper bounds for each supplier (capacity constraints)
x_bounds = [(0, cap) for cap in capacities]

# Solve the linear optimization problem
result = linprog(c, A_eq=[A[-1]], b_eq=[b[-1]], A_ub=A[:-1], b_ub=b[:-1], bounds=x_bounds, method='highs')

# Extract results
supplier_quantities = result.x
total_cost = sum(supplier_quantities[i] * costs[i] for i in range(len(supplier_quantities)))
average_cost = total_cost / total_demand

# Prepare data for display
output_data = pd.DataFrame({
    'Supplier': [f'Supplier {i+1}' for i in range(len(supplier_quantities))],
    'Quantity (mtons)': supplier_quantities,
    'Cost ($/ton)': costs,
    'Total Cost ($)': [q * costs[i] for i, q in enumerate(supplier_quantities)]
})
output_data.loc['Total'] = output_data[['Quantity (mtons)', 'Total Cost ($)']].sum()
output_data.loc['Total', 'Supplier'] = 'Total'
output_data.loc['Total', 'Cost ($/ton)'] = average_cost

#import ace_tools as tools; tools.display_dataframe_to_user(name="Coal Procurement Optimization Results", dataframe=output_data)
print (output_data)
total_cost, average_cost


         Supplier  Quantity (mtons)  Cost ($/ton)  Total Cost ($)
0      Supplier 1              12.5     49.500000          618.75
1      Supplier 2             600.0     50.000000        30000.00
2      Supplier 3              72.0     61.000000         4392.00
3      Supplier 4               0.0     63.500000            0.00
4      Supplier 5             120.0     66.500000         7980.00
5      Supplier 6               0.0     71.000000            0.00
6      Supplier 7             420.5     72.500000        30486.25
7      Supplier 8               0.0     80.000000            0.00
Total       Total            1225.0     59.981224        73477.00


(73477.0, 59.98122448979592)